## Threads
There are two primary ways to create a thread. 
1. Extend the `Thread` class and override the run method.

In [ ]:
// Worker cannot inherit some functionality from other classes
public class Worker extends Thread {
    @Override
    public void run(){
        System.out.println("Worker thread running");
    }
}

Worker worker = new Worker();

2) The preferred way is to pass a `Runnable` instance to `Thread` constructor:

In [ ]:
Runnable runnable = new Runnable{
    public void run(){
        System.out.println("Worker thread running");
    }
}

Thread worker = new Thread(runnable);

**Starting a thread:** use the `start` method of the `Thread` class. The `isAlive` method can tell if the thread is currently running or not. `isAlive` returns false if the thread has not been started or the thread has been terminated. By default, the thread is named *Thread-N*.

**Stopping a thread:** is not as straight forward as starting it. The only way to terminate a thread is to arrange for its `run` method to complete. The thread class has a `stop` method, but it has been deprecated. It has been deprecated because stopping a thread releases all the locks it may have. If the thread was inside a critical section when stopped, it would lead to corrupted state. Other threads which access that critical section might be working with corrupted state.

Thread throwing an unchecked exception can also terminate the thread. 

Two approaches for stopping a thread: 
1. setting a flag

In [ ]:
public class Worker extends Thread {
    // done must be set as volatile for this to work
    private volatile boolean done = false;
    
    @Override
    public void run(){
        while(!done){
            // Do something
        }
    }
    
    // Other methods
}

&emsp; An external code can stop this thread by setting done as true. But the thread will only stop when the execution reaches the top of the loop (and not immediately).

2. Using interrupts: we can call `interrupt` method of any thread. It has two effects. First, it sets internal interrupted flag of the thread to true. Second, it causes any blocking method like `sleep`, `join`, `wait` to stop blocking and throw `InterruptedException`.

In [ ]:
public class Worker extends Thread{
    @Override
    public void run(){
        while(!isInterrupted()){
            // Do something
        }
    }
    
    // Other methods
}

Worker worker = new Worker();
worker.start();
worker.interrupt();

&emsp; If we are using `Runnable` interface, the same can be achieved using the static `interrupted()` method (equivalent to `Thread.currentThread().isInterrupted()`

&emsp; As mentioned above, blocking methods also respond to interruptions:

In [ ]:
public class InterruptionDemo {
    public static void main(String[] args) throws InterruptedException {
       ParentThread parent = new ParentThread();
       parent.start();

       Thread.sleep(1000);
       parent.interrupt(); // parent thread is interrupted,
                           // this has no effect on child thread
                           // which continues running
    }
}

class ParentThread extends Thread {
    @Override
    public void run() {
       ChildThread child = new ChildThread();
       child.start();

        try {
            child.join();  // waiting till child thread completes execution
        } catch (InterruptedException e) {
          System.out.println("ParentThread interrupted, exiting");
        }
    }
}

class ChildThread extends Thread {
    @Override
    public void run() {
       while(!isInterrupted()) {
          System.out.println("ChildThread running...");
          try {
	          // Depending upon when interrupt is called on this thread
	          // it will wait for anywhere between 0 to 1000ms.
	          Thread.sleep(1000);
          } catch(InterruptedException e) {
             System.out.println("ChildThread interrupted, exiting");
             return;
          }
       }
    }
}

**Pausing a thread:** the Thread class has `suspend` and `resume` methods which have been deprecated (due to possibility of deadlock). It is still possible for a thread to suspend its own execution for a specified amount of time using `sleep` method. The least amount of time a thread can sleep on most Java implementations is 20 to 50 ms.

In [6]:
public class DelayedRandomGenerator extends Thread {
    public void run() {
        Random random = new Random();
        // If DelayedRandomGenerator implemented Runnable instead, use Thread.currentThread().isInterrupted()
        while (!isInterrupted()) {
            System.out.print(random.nextInt() + "\t");
            try {
                sleep(200);
            } catch (InterruptedException e) {
                break;
            }
        }
    }
}

DelayedRandomGenerator generator = new DelayedRandomGenerator();
generator.start();

try {
    Thread.sleep(1000);
} catch (InterruptedException e) {
    System.exit(0);
}        

generator.interrupt();

1152488192	2140810343	1285862342	1650368227	-1184878083	1039227028	278517272	-1543200418	401414008	-531716027	

**Thread Lifecycle:** the `ThreadState` class defines some granular states a thread can be in:
- ALLOCATED: memory allocated, not intialised
- INITIALISED: initialised, not started
- RUNNABLE: has been started, not necessarily running
- MONITOR_WAIT: waiting for monitor entry
- CONDVAR_WAIT: waiting on `Condition`
- OBJECT_WAIT: waiting on `wait()`
- BREAKPOINTED: suspended at breakpoint
- SLEEPING: `Thread.sleep()`
- ZOMBIE: all done, not reclaimed yet

## Synchronization
Consider the following example:

In [ ]:
class Counter {
    private int count = 0;

    public void increment() {
        count++;
    }

    public int getCount() {
        return count;
    }
}

public class Demo {
    public static void main(String[] args) throws InterruptedException {
        Counter counter = new Counter();

        Thread t1 = new Thread(() -> {
            for (int i = 0; i < 1000; i++) counter.increment();
        });

        Thread t2 = new Thread(() -> {
            for (int i = 0; i < 1000; i++) counter.increment();
        });

        t1.start();
        t2.start();
        t1.join();
        t2.join();

        System.out.println("Final count: " + counter.getCount());
    }
}

The above example suffers from *race condition* where two or more threads access shared data simultaneously, and the final outcome depends on the order of execution.

The statement `count++` is not atomic, it is composed of multiple steps:
- `temp = count` load value of `count` from memory
- `temp = temp + 1;` increment value of `count`
- `count = temp` store final value back to memory

This is an example of *read-modify-writ*e operation. When two threads operate on count simultaneously, the above listed steps may interleave. Which results in a final value other than the expected 2000.

In addition to *read-modify-write* race condition, we can have *check-then-act race* condition. Below example of lazy initialization explains it:

In [ ]:
public class Singleton {
    private Singleton instance = null;
    
    public Singleton getInstance() {
        if (instance == null)
            instance = new Singleton();
        return instance;
    }
}

The above example has race condition because two threads can view instance variable to be null at the same time, thereby two different instance objects get created, which isn't what we would want.

One way to solve such race conditions is by serializing access to the method using `synchronized` keyword. When a method is marked as `synchronized`, any thread that tries to invoke it must first obtain lock. Only one thread can have access to the lock at any given time ensuring exclusive access.

In [ ]:
class Counter {
    private int count = 0;

    // The lock object in this case is `this`. Every object can implicitly act as a lock for purposes of synchronization; 
    // these built-in locks are called intrinsic locks or monitor locks.
    public synchronized void increment() {
        count++;
    }

    public synchronized int getCount() {
        return count;
    }
}

`synchronized` can also be applied to `static` methods. In that case the `Class` object would act as the lock object.

We also have `synchronized` blocks:

In [ ]:
private int lastInput;
private int[] lastFactors;

public int[] getFactors(int input) {
    int[] factors = null;

    synchronized(this) { // Accepts object to act like lock object
        if(input == lastInput) {
            factors = lastFactors.clone();
        }
    }

    if(factors == null) {
        factors = calculateFactors(input);
        synchronized(this) {
            lastInput = input;
            lastFactors = factors.clone();
        }
    }

    return factors;
}

Acquiring and releasing a lock has some overhead, so it is undesirable to break down synchronized blocks too far. Avoid holding locks during lengthy computations or operations at risk of not completing quickly such as network or console I/O.

If an object has two independent state variables, it better to use to different lock objects such that two different threads can access two states concurrently.

In [ ]:
class SharedObject {
    private long c1 = 0;
    private long c2 = 0;
    private Object o1 = new Object();
    private Object o2 = new Object();

    public void incrementC1() {
        synchronized(o1){
            this.c1++;
        }
    }

    public void incrementC2() {
        synchronized(o2){
            this.c2++;
        }
    }
}

### Explicit Locks
`Lock` interface and associated implementations provide more control over locking mechanism. Explicit locks provide the same memory visibility guarantees as `synchronized`. The interface is defined as:

In [ ]:
public interface Lock {
    void lock();
    void lockInterruptibly() throws InterruptedException;  // thread waiting to acquire lock can be interrupted

    // Acquires the lock if it is available and returns immediately with the value true. 
    // If the lock is not available then this method will return immediately with the value false.
    boolean tryLock();
    boolean tryLock(long var1, TimeUnit var3) throws InterruptedException;

    void unlock();
     
    // ...
}

`Lock` is typically used in the following manner:

In [ ]:
Lock lock = new ReentrantLock();

public void someMethod() {
    try {
        lock.lock(); // acquire lock
        // critical section
    } finally {
        lock.unlock(); // release lock
    }
}

// Essentially the same as
public synchronized void someMethodSynch() {
    // critical section
}

**ReentrantLock:** an implementation of `Lock` is very similar to `synchronized`. A reentrant lock lets the same thread acquire the same lock more than once without blocking itself. Some additional methods provided by `ReentrantLock` are:

In [ ]:
public class ReentrantLock implements Lock {
	public int getHoldCount() {
		// ...
	}
	
	public boolean isLocked() {
		// ...
	}
	
	public boolean isHeldByCurrentThread() {
		// ...
	}
	
	// other methods
}

`ReentrantLock` keeps track of the number of recursive acquisitions of the lock, finally freeing the lock upon exiting the first method (or block) that acquired the lock. Calls to `lock` increment the count and calls to `unlock` decrement this count. The lock is not freed until the lock count reaches zero. This arrangement prevents deadlock as the below example explains:

In [ ]:
public synchronized void outer() {
    inner(); // also synchronized on same object
}

public synchronized void inner() {
    // ...
}

Without reentrancy:
1. `outer()` acquires the lock.
2. It calls `inner()`, which tries to acquire the same lock.
3. The lock is already held by the same thread → it waits for itself → self-*deadlock*.

The `getHoldCount()` method returns the number of acquisitions that the current thread has made on the lock. A return value of zero means that the current thread does not own the lock, it does not mean that the lock is free. To determine if the lock is free and not acquired by any thread, the `isLocked()` method may be used.

The `ReentrantLock` class allows the developer to request that locks be granted fairly (first come first serve basis; note that definition of fairness can be different) by passing true to the constructor like `new ReentrantLock(true)`. This means that there is no possibility of starvation. This is different from `synchronized` where any waiting thread can acquire lock.

### Read-Write Lock
In the scenario where the lock needs to be acquired for a long duration for read purpose, it makes sense to allow other threads to read simultaneously. We would still only allow one thread to write and that too only when no other threads are reading the data.

Java provides `ReadWriteLock` and its implementation `ReentrantReadWriteLock` to:
- allow multiple readers to hold the lock simultaneously, as long as there are no writers.
- only one writer to hold the lock, and no readers when a writer is active.

In [ ]:
public interface ReadWriteLock {
    Lock readLock( );
    Lock writeLock( );
}
public class ReentrantReadWriteLock implements ReadWriteLock {
    public ReentrantReadWriteLock( );
    // Lock is granted close to FIFO order
    public ReentrantReadWriteLock(boolean fair);
    public Lock writeLock( );
    public Lock readLock( );
}

Below example illustrates usage of read write lock:

In [ ]:
class Configuration {
    private final Map<String, String> config = new HashMap<>();
    private final ReadWriteLock lock = new ReentrantReadWriteLock();
    private final Lock readLock = lock.readLock();
    private final Lock writeLock = lock.writeLock();

    public String getConfig(String key) {
        readLock.lock();
        try {
            return config.get(key);
        } finally {
            readLock.unlock();
        }
    }

    public void reloadConfig() {
        writeLock.lock();
        try {
            config.clear();
            // fetch config from somewhere
            config.putAll(...);
        } finally {
            writeLock.unlock();
        }
    }
}

Reader writer locks have different support for condition variables:
- we can obtain a `Condition` variable related to the write lock by calling the `newCondition( )` method
- calling `newCondition( )` on a read lock generates an `UnsupportedOperationException`

Also:
- threads that own the write lock can also acquire the read lock
- threads that own the read lock cannot acquire the write lock

In [ ]:
writeLock.lock();
try {
    // This is allowed
    readLock.lock();
    try {
        System.out.println("Thread can read while holding write lock");
    } finally {
        readLock.unlock();
    }
} finally {
    writeLock.unlock();
}

In [ ]:
readLock.lock();
try {
    // This will block forever
    writeLock.lock();
    try {
        System.out.println("Upgraded to write lock");
    } finally {
        writeLock.unlock();
    }
} finally {
    readLock.unlock();
}

### Deadlock
Is a scneario where two or more threads are waiting for each other’s resources (locks), and none can proceed. As an example, if we replace `volatile` variable with `synchronized` method to set done as `true`, we will be stuck with deadlock:

In [ ]:
public class Worker extends Thread {
    private boolean done = false;
    
    @Override
    public synchronized void run() { // As long as current thread is running it has exclusive access
        while(!done){                // to the lock and other thread can't execute setDone()
            // Do something
        }
    }
    
    public synchronized void setDone() {
        done = true;
    }
}

Another example involving two threads and two locks:

In [ ]:
private final A a = new A();
private final B b = new B();

// Method 1 acquires locks in a -> b order
public void method1() {
    synchronized (a) {
        System.out.println(Thread.currentThread().getName() + " locked A");
        try { Thread.sleep(100); } catch (InterruptedException e) {}
        synchronized (b) {
            System.out.println(Thread.currentThread().getName() + " locked B");
        }
    }
}

// Method 2 acquires locks in b -> a order
public void method2() {
    synchronized (b) {
        System.out.println(Thread.currentThread().getName() + " locked B");
        try { Thread.sleep(100); } catch (InterruptedException e) {}
        synchronized (a) {
            System.out.println(Thread.currentThread().getName() + " locked A");
        }
    }
}

/*
If two threads are involved, t1 invokes method1 and t2 invokes method2,
t1 will be stuck acquiring lock b, whereas t2 will be stuck acquiring lock a.
To mitigate this type of issue acquire locks in the same order.
*/

Deadlock can also happen if one thread waits for itself to finish:

In [ ]:
public class JoinDeadlock {
    public static void main(String[] args) throws InterruptedException {
        Thread t1 = new Thread(() -> {
            try {
                System.out.println("T1 waiting for main...");
                Thread.currentThread().join(); // waiting for itself!
            } catch (InterruptedException e) {}
        });

        t1.start();
        t1.join();  // main waiting for t1

        System.out.println("Done");
    }
}

## Thread Notification
Thread notification works on the idea that a thread needs certain condition to be exist and that some other thread will create that condition. Three methods of `Object` class provide this mechanism:
- `wait()` : waits for a condition to occur. Must be called from within a `synchroized` method/block. Calling `wait` gives up the lock.
- `wait(long timeout)`: similar to the first except that it comes back into contention for the lock acquisition after the timeout.
- `notify()`: signals a random waiting thread (guarded by the same lock) to that the condition has occurred. This also must be called from within `synchronized` block/method. Calling `notify` doesn’t mean lock is given up.
- `notifyAll()` : notifies all waiting threads (guarded by the same lock) instead of an arbitrary thread. The notified threads take turn to acquire lock and continue from `wait`

Since any Java object can be used as lock, it makes sense that the `Object` class contained these methods.

As mentioned above, it is imperative that `wait` and `notify` methods are called from within `synchronized` - this is to prevent race condition associated with *wait-and-notify* mechanism:
1. Thread A tests a condition and discovers it must wait.
2. Thread B sets the condition and calls `notify()` to inform A to resume execution. Because A is not yet waiting, nothing happens.
3. Thread A waits, by calling `wait()`.
4. Because of the prior `notify()` being missed, A waits indefinitely.

It is mandatory for `wait` and `notify` to hold the locks for the object they are operating on. The `wait` method releases the lock prior to waiting and reacquires the lock prior to returning from the wait method. Typical *wait-notify* construct looks like:

In [ ]:
synchronized (lock) {
    while (!condition) {
        lock.wait();   // releases lock and waits
    }
    // ... do work after condition true
}

synchronized (lock) {
    condition = true;
    lock.notify();     // or notifyAll()
}

If a thread receives a notification, it is not guaranteed that the condition is set correctly. Prior to calling the `wait` method, thread should test for the condition while holding the lock. Upon returning from the `wait` method the thread should retest the condition (to check whether it should wait again). Why? Suppose three threads A, B and C:
1. Thread A calls a method that acquires the `synchronization` lock.
2. Thread A examines a state flag and determines that the data is not in the desired state.
3. Thread A calls the `wait()` method, which frees the lock.
4. Thread B calls a method that acquires the same `synchronization` lock.
5. Thread C calls a method that blocks waiting for the lock.
6. Thread B sets the state flag and calls the `notifyAll()` method.
7. Thread B finishes its method and frees the lock.
8. Thread C acquires the lock and proceeds to process the data; it sees that the data is in the desired state, so it processes the data and resets the state flag.
9. Thread C exits without needing to wait.
10. Thread A receives the notification and wakes up.

Threads that are waiting can be considered as consumers - there is no guarantee that when the thread wakes up the data has not been processed by another consumer. When the thread wakes up, it cannot assume that the condition it has been waiting for would still be true. Waiting thread must provide an option to check the state and return back to waiting state in case the notification has already been handled. This is something that calls to `wait()` inside a for loop can achieve.

In [ ]:
class Data {
    private String data;
    private Object lock = new Object();
    private boolean sent = true;

    public void send(String data) {
        synchronized (lock) {
            while (!sent) {
                try {
                    // Notice that wait is called on the lock object
                    lock.wait();
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }

            sent = false;
            this.data = data;

            // Notify is also called on the same lock
            lock.notifyAll();
        }
    }

    public String receive(){
        synchronized (lock) {
            while (sent) {
                try {
                    lock.wait();
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }

            sent = true;
            lock.notifyAll();

            return this.data;
        }
    }
}

*Wait and Notify* mechanism forms the basis of producer consumer pattern:

In [ ]:
class Common {
    private int number;
    private boolean available = false;

    public synchronized void setNumber(int number) {
        // Producer waits if the data is already available
        while (available) {
            try {
                wait();
            } catch (InterruptedException e) {
                e.printStackTrace();
                return;
            }
        }

        available = true;
        this.number = number;
        System.out.println("Number " + this.number + " set");

        notifyAll(); // If we miss this all consumers will continue
                  // waiting. Here we notify all waiting consumers that
                  // that new data is ready
    }

    public synchronized int getNumber() {
        // Producer has not produced new data, so better wait
        while (!available) {
            try {
                wait();
            } catch (InterruptedException e) {
                e.printStackTrace();
                return -1;
            }
        }

        available = false;
        notifyAll(); // Notify producer that data has been consumed
                     // they can stop waiting (after this method
                     // has been executed)

        System.out.println("Number " + this.number + " read");
        return this.number;
    }
}

class Producer extends Thread {
    private Common common;

    public Producer(Common common) {
        this.common = common;
    }

    @Override
    public void run() {
        // Produce even numbers sequentially
        for (int i = 0; i < 100; i++) {
            if (i % 2 == 0) {
                common.setNumber(i);
            }
        }
    }
}

class Consumer extends Thread {
    private Common common;

    public Consumer(Common common) {
        this.common = common;
    }

    @Override
    public void run() {
        // Consume all generated numbers
        for (int i = 0; i < 50; i++) {
            int i = common.getNumber();
            if(i == -1) break;
        }
    }
}

### Condition
A `Condition` is `Lock`'s equivalent for `wait` and `notify` constructs. It is defined as:

In [ ]:
public interface Condition {
	void await() throws InterruptedException; // similar to wait()
	void awaitUninterruptibly(); // no equivalent
    void await
	
	void signal(); // similar to notify
	void signalAll(); // similar to notifyAll
	
	// other methods
}

When execution reaches `await`, lock is released for other thread to acquire. Execution is resumed when the condition is signalled using `signal` or `signalAll`. Note that *awaiting* thread is woken up only after the signalling thread gives up lock.

You need to hold lock for which you invoke `signal` or `await`. It is also possible to create more than one `Condition` variable for a given lock object. Example usage of `Condition`:

In [ ]:
class Common {
    private boolean available = false;
    private final Lock lock = new ReentrantLock();
    private final Condition condition = lock.newCondition();

    public void produce() {
        lock.lock();
        try {
            while(available) {
                condition.await();
            }
            
            available = true;
            condition.signalAll();
        } catch (InterruptedException e) {
            e.printStackTrace();
        } finally {
            lock.unlock();
        }
    }

    public void consume() {
        lock.lock();
        try {
            while(!available) {
                condition.await();
            }
            
            available = false;
            condition.signalAll();
        } catch (InterruptedException e) {
            e.printStackTrace();
        } finally {
            lock.unlock();
        }
    }
}

## Atomic Variables
Java provides a set of classes that allow multiple operations to be treated atomically. This capability is beyond what can be achieved using just `volatile`. For example, `++` operator with a volatile variable doesn't really work because it is comprised of multiple instructions. With atomic variable like `AtomicInteger` this can be achieved without synchronization.

Java provides the following classes:
- `AtomicInteger`: an `int` value
- `AtomicLong`: a `long` value
- `AtomicBoolean`: a single `boolean` value
- `AtomicReference<V>`: holds reference to an object 
- `AtomicIntegerArray`: array of `int`
- `AtomicLongArray`: array of `long`

### `AtomicInteger`
Contains the following methods (not the full list):
<div style="display: inline-block">

| Method                                  | Description                                                              |
| --------------------------------------- | ------------------------------------------------------------------------ |
| `get()`                                 | Returns the current value                                                |
| `set(int newValue)`                     | Sets to new value (not atomic with respect to other reads)               |
| `getAndSet(int newValue)`               | Atomically sets to `newValue` and returns the **old** value              |
| `compareAndSet(int expect, int update)` | CAS — sets to `update` if current value equals `expect`; returns boolean |
| `getAndIncrement()`                     | Atomically returns current value, then increments by 1                   |
| `incrementAndGet()`                     | Increments by 1, then returns new value                                  |
| `getAndDecrement()`                     | Returns current, then decrements by 1                                    |
| `decrementAndGet()`                     | Decrements, then returns new value                                       |
| `addAndGet(int delta)`                  | Adds delta, then returns new value                                       |
| `getAndAdd(int delta)`                  | Returns current, then adds delta                                         |

In [2]:
import java.util.concurrent.atomic.AtomicInteger;

AtomicInteger counter = new AtomicInteger();  // By default set to 0
counter.incrementAndGet();     // 1
counter.addAndGet(5);          // 6
counter.compareAndSet(6, 10);  // true. Means if(counter == 6) counter = 10;
counter.get();                 // 10

10

### `AtomicLong`
Contains the following methods:
<div style="display: inline-block">

| Method                                           | Description                                   |
| ------------------------------------------------ | --------------------------------------------- |
| `get()`, `set(long)`                             | Get or set current value                      |
| `getAndSet(long newValue)`                       | Atomically sets and returns old value         |
| `compareAndSet(long expect, long update)`        | CAS for long                                  |
| `getAndIncrement()`, `incrementAndGet()`         | ++                                            |
| `getAndDecrement()`, `decrementAndGet()`         | --                                            |
| `addAndGet(long delta)`, `getAndAdd(long delta)` | Add operations                                |

### `AtomicBoolean`
Contains the following methods (not the full list):
<div style="display: inline-block">

| Method                                          | Description                                        |
| ----------------------------------------------- | -------------------------------------------------- |
| `get()`                                         | Returns the current value                          |
| `set(boolean newValue)`                         | Sets to new value                                  |
| `compareAndSet(boolean expect, boolean update)` | CAS for boolean                                    |
| `getAndSet(boolean newValue)`                   | Atomically sets to new value and returns old value |

In [3]:
import java.util.concurrent.atomic.AtomicBoolean;

AtomicBoolean flag = new AtomicBoolean();
flag.compareAndSet(false, true);
flag.get();

true

### `AtomicReference<V>`
Holds a reference to an object atomically — useful for lock-free state updates. Contains the below listed methods:
<div style="display: inline-block">

| Method                                  | Description                                       |
| --------------------------------------- | ------------------------------------------------- |
| `get()`                                 | Returns current reference                         |
| `set(V newValue)`                       | Sets new reference                                |
| `compareAndSet(V expect, V update)`     | Atomically replaces value if equals expected      |
| `getAndSet(V newValue)`                 | Atomically sets and returns old reference         |

In [4]:
import java.util.concurrent.atomic.AtomicReference;

AtomicReference<String> ref = new AtomicReference<>("INIT");
ref.compareAndSet("INIT", "READY");  // true
ref.get();  // READY

READY

### `AtomicIntegerArray`
Is an array of int values where each element supports atomic updates. Contains:
<div style="display: inline-block">

| Method                                                                | Description                        |
| --------------------------------------------------------------------- | ---------------------------------- |
| `length()`                                                            | Returns array length               |
| `get(int index)`                                                      | Gets value at index                |
| `set(int index, int newValue)`                                        | Sets new value                     |
| `getAndSet(int index, int newValue)`                                  | Atomically sets, returns old value |
| `compareAndSet(int index, int expect, int update)`                    | CAS on element                     |
| `getAndIncrement(int index)`, `incrementAndGet(int index)`            | ++ element                         |
| `getAndAdd(int index, int delta)`, `addAndGet(int index, int delta)`  | Add operation                      |

In [5]:
import java.util.concurrent.atomic.AtomicIntegerArray;

AtomicIntegerArray scores = new AtomicIntegerArray(3);  // array of length 3
scores.set(0, 10);         // Set 0th index to 10
scores.incrementAndGet(0); // 11
scores.length();

3

Example usage of atomic variable:

In [ ]:
public void replaceObservable(Observable newObservable) {
    Observable oldObservable;

    if (newObservable != null) {
        newObservable.addObserver(this); // this represents current Observer
    }

    oldObservable = observable.getAndSet(newObservable); // observable is an instance AtomicReference
    if (oldObservable != null) {
        oldObservable.removeObserver(this);
    }
}

/*
Synchronized version:
public synchronized void replaceObservable(Observable newObservable) {
    if (observable != null) {
        observable.removeObserver(this);
    }

    observable = newObservable;
    if (observable != null) {
        observable.addObserver(this); // this represents current Observer
    }
}
*/

Replacing `synchronized` version with atomic variable may also involve introducing endless loops. For example:

In [ ]:
/*
private int counter = 0;
public synchronized int increment() {
    counter++;
}
*/

private AtomicInteger counter = new AtomicInteger();
// Note: this is just an example, it could be easily replaced with
// call to counter.incrementAndGet(); 
public void increment() {
    int oldValue, newValue;
    do {
        oldValue = counter.get();
        newValue = oldValue + 1;
    } while(!counter.compareAndSet(oldValue, newValue));
    // Loop ensures that no other thread interfered with
    // calculations done in the do loop. If another thread
    // had also executed do block before current thread,
    // then the while loop condition would not have been true
}

Another similar example where we have to introduce a *CAS loop*:

In [ ]:
/*
public synchronized boolean withdraw(int amount) {
    if (balance >= amount) {
        balance -= amount;
        return true;
    }
    
    return false;
}
*/

private AtomicInteger balance = new AtomicInteger();
public boolean withdraw(int amount) {
    int oldBalance, newBalance;
    do {
        oldBalance = balance.get();
        newBalance = oldBalance - amount;
        if(newBalance < 0) {
            return false;
        }
    } while(!balance.compareAndSet(oldBalance, newBalance));
    
    return true;
}

// Or
public boolean withdraw(int amount) {
    int oldBalance, newBalance;
    while(true) {
        oldBalance = balance.get();
        newBalance = oldBalance - amount;
        if(newBalance < 0) {
            return false;
        } else if(balance.compareAndSet(oldBalance, newBalance)) {
            return true;
        }                
    }
}

Another example showing inserting a node at the start of a linked list:

In [ ]:
/*
public synchronized void insert(int value) {
    Node newNode = new Node(value);
    newNode.next = head;
    head = newNode;
}
*/

private AtomicReference<Node> head = new AtomicReference<>(null);
public void insert(int value) {
    Node oldHead;
    Node newNode = new Node(value);
    do {
        oldHead = head.get();
        newNode.next = oldHead;        
    } while(!head.compareAndSet(oldHead, newNode));
}

When we replace a `synchronized` block with atomics, you remove mutual exclusion and rely on optimistic concurrency via *CAS loops*. That means:
- We must manually retry until success `(while(!compareAndSet(...)))`.
- Under continuous interference (high contention), this retry loop can spin forever.
- So the system is still thread-safe, but not guaranteed to make progress — that’s called *lock-free* but not *wait-free*.

## ThreadLocal
Enables us to save data specific to a thread. Data stored in `ThreadLocal` is not shared between threads. The class looks like:

In [ ]:
public class ThreadLocal<T> {
    protected T initialValue();
    public T get();
    public void set(T value);
    public void remove();
}

Typically we would subclass `ThreadLocal` and override the `initialValue` method:

In [ ]:
static ThreadLocal<Map<Integer, Integer>> maps = new ThreadLocal<> {
    protected Map<String, String> initialValue() {
        return new HashMap<>();
    }
};

public int calculate(int input) {
    Map<Integer, Integer> map = maps.get();
    Integer result = map.get(input);
    if(result != null) {
        return result;
    }

    result = doCalculate(input);
    map.put(input, result);
    return result;
}